In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/francoisderyckel/Documents/quant-sandbox/python':
  os.chdir(r'/Users/francoisderyckel/Documents/quant-sandbox/python')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/_bootstrap.py": 1701988380.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/_bootstrap_external.py": 1701988380.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/zipimport.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/codecs.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/encodings/aliases.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/encodings/__init__.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/encodings/utf_8.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/abc.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/io.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/types.py": 1701988379.0, "/Lib

/var/folders/4k/sh_jysz50kq5c92jvljhs06c0000gp/T/ipykernel_96827/3484563495.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [2]:
#| label: loading-packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf 
import mplfinance as mpl

from datetime import date
from dateutil.relativedelta import relativedelta

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report

In [3]:
#| label: loading-data
#| output: false

# downloading the data and storing it in a dataframe
rio = yf.download('RIO')
df = pd.DataFrame(rio)

[*********************100%%**********************]  1 of 1 completed

In [4]:
#| label: initial-eda

# get to know df variables
print('The shape of dataframe is:', df.shape)
print(df.head())

# checking for missing values. 
df.isnull().sum()

The shape of dataframe is: (8545, 6)
                Open      High       Low     Close  Adj Close  Volume
Date                                                                 
1990-06-28  10.09375  10.09375   9.96875   9.96875   1.916216  176400
1990-06-29  10.03125  10.06250  10.00000  10.06250   1.934236   69200
1990-07-02  10.00000  10.03125  10.00000  10.03125   1.928229   62000
1990-07-03  10.03125  10.06250  10.03125  10.06250   1.934236   29600
1990-07-05   9.71875   9.71875   9.65625   9.68750   1.862152   31200


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [5]:
#| echo: false

df.loc['2019-01-01':].describe().round(2)

,Open,High,Low,Close,Adj Close,Volume
count,1362.00,1362.00,1362.00,1362.00,1362.00,1362.00
mean,65.09,65.64,64.52,65.11,52.64,2920743.54
std,10.47,10.54,10.37,10.49,12.29,1306051.29
min,36.10,37.21,35.35,36.42,25.78,378200.00
25%,58.23,58.74,57.69,58.29,39.82,2028000.00
50%,63.50,64.03,63.07,63.65,55.84,2709350.00
75%,71.67,72.22,71.02,71.77,63.00,3548800.00
max,95.23,95.97,93.65,94.65,73.61,10406600.00


In [6]:
#| label: plot01

# Checking only over the last 5 years of data 
start_date = date.today() + relativedelta(days=int(np.round(-5*365)))
end_date = date.today()

# first vizualization 
plt.plot(df['Adj Close'].loc[start_date:end_date])
plt.show()

<Figure size 1650x1050 with 1 Axes>

In [7]:
#| label: candlestick-plot01
#| echo: false

df_last18months = df.loc[start_date:end_date]

fig, axlist = mpl.plot(
  df_last18months, 
  type = 'candle', 
  style='binance', 
  figratio = (15, 7), 
  returnfig = True
)

fig = fig.suptitle(
  f"$RIO stock between {start_date} and {end_date}", 
  y=.95, 
  fontsize=15, 
  weight = 'semibold', 
  style = 'normal'
)

mpl.show()

/Users/francoisderyckel/.virtualenvs/quant/lib/python3.12/site-packages/mplfinance/_arg_validators.py:84: UserWarning:




            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.




<Figure size 1232.14x575 with 2 Axes>

In [8]:
#| label: feature-eng-1

# getting returns. We shift it as it constitutes the base of what we have to predict. 
df['ret_1d_target'] = np.log(df['Adj Close'].shift(-1) / df['Adj Close'])
df['ret_1d_shifted'] = (np.log(df['Adj Close']).diff(1)).shift(1)
df.head()

# creating variables based on intra day. 
df['o-c'] = np.log(df.Open/df.Close).shift(1)   # Same day Open-Close
df['h-l'] = np.log(df.High/df.Low).shift(1)     # Same day High-Low
df['o-o'] = np.log(df.Open/df.Open.shift(1))    # Open-Previous day Open
df['o-prevC'] = np.log(df.Open/df.Close.shift(1)) # Open - Previous day Close

newcol = {}     # avoid df fragmentation warnings

# creating our lag variables
for lags in range(1, 17, 1): 
  newcol['ret_1d_lag' + str(lags) + 'd'] = df.ret_1d_shifted.shift(lags)
  newcol['o-c-lag' + str(lags) + 'd'] = df['o-c'].shift(lags)
  newcol['o-o-lag' + str(lags) + 'd'] = df['o-o'].shift(lags)
  newcol['h-l-lag' + str(lags) + 'd'] = df['h-l'].shift(lags)
  newcol['o-prevC-lag' + str(lags) + 'd'] = df['o-prevC'].shift(lags)

# Creating momentum variables
for days in range(2, 61, 3): 
  newcol['ret_'+ str(days)] = df.ret_1d_shifted.rolling(days).sum()
  newcol['sd_' + str(days)] = df.ret_1d_shifted.rolling(days).std()

# creating SMA percentage
for lags in range(2, 61, 4): 
  newcol['sma_'+str(lags)+'d']=df['Adj Close'].rolling(window=lags).mean()
  newcol['sma_'+str(lags)+'d']=np.log(df['Adj Close']/newcol['sma_'+str(lags)+'d'])
  newcol['sma_'+str(lags)+'d']=newcol['sma_'+str(lags)+'d'].shift(1)

# creating EMA percentage
for lags in range(3, 10, 2): 
  newcol['ema_'+str(lags)+'d']=df['Adj Close'].ewm(span=lags, adjust=False).mean() 
  newcol['ema_'+str(lags)+'d']=np.log(df['Adj Close']/newcol['ema_'+str(lags)+'d'])
  newcol['ema_'+str(lags)+'d']=newcol['ema_'+str(lags)+'d'].shift(1)

df_newcol = pd.DataFrame(newcol)
df = pd.concat([df, df_newcol], axis = 1)

print(df.shape)

(8545, 151)


In [9]:
#| label: create-target

# slicing the dataframe to get all data after 2019 (close to last 5 years)
df_new = df.loc['2019-01-01':].copy()
df_new.shape

# Step 2: Calculate the median of the targetcolumn in the filtered data
median_return = df_new['ret_1d_shifted'].dropna().median()
print(f"Median Return: {np.round(median_return, 6)}")

# Step 3: Create the 'target' column based on the median return
df_new['target'] = np.where(df_new['ret_1d_target'] > median_return, 1, 0)
df_new.shape
df_new['target'].value_counts(normalize = True).round(4) * 100
df_new.dropna(inplace = True)

# Step 4: Remove all unwanted columns and target variables
df3 = df_new.drop(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'target', 'ret_1d_target'], axis = 1).copy()

# checking again size of data frame
print(f"Final data set contais {df3.shape[1]} features")

# setting up our inputs and output dataframe
X = df3
y = df_new['target']

Median Return: 0.001433
Final data set contais 144 features


In [10]:
#| label: training-testing

# put shuffle to False as we are dealing with a timeseries where order of data matter.
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

print('Shape of x_train:', x_train.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of x_test:', x_test.shape)
print('Shape of y_test:', y_test.shape)
print(f"Size of training set is {len(x_train)} and size of testing set is {len(x_test)}")

Shape of x_train: (1088, 144)
Shape of y_train: (1088,)
Shape of x_test: (273, 144)
Shape of y_test: (273,)
Size of training set is 1088 and size of testing set is 273


In [11]:
#| label: correlations

corr_matrix = df3.corr()

# unstack the get the list of all pairs and put it in dataframe
corr_p = corr_matrix.unstack()      
corr_p_df = pd.DataFrame(corr_p,columns=['Correlation']).reset_index()

# Rename the columns for clarity
corr_p_df.columns = ['feature_1', 'feature_2', 'Correlation']

# Remove self-correlations
corr_p_df = corr_p_df[corr_p_df['feature_1'] != corr_p_df['feature_2']]

# Drop duplicate pairs 
corr_p_df['abs_corr'] = corr_p_df['Correlation'].abs()
corr_p_df = corr_p_df.sort_values(by='abs_corr', ascending=False).drop_duplicates(subset=['Correlation']).drop(columns=['abs_corr'])

# Sort the pairs by the absolute value of the correlation
sorted_corr_p = corr_p_df.sort_values(by='Correlation', ascending=False)

# Select the top 20 pairs
top_10_corr_pairs = sorted_corr_p.head(10)

print(f"There are {len(corr_p_df[corr_p_df['Correlation'].abs() > 0.8])} highly correlated pairs (above 0.8) from the {len(sorted_corr_p)} possible combination.")

There are 428 highly correlated pairs (above 0.8) from the 10296 possible combination.


In [12]:
#| label: correlations_table
#| echo: false

top_10_corr_pairs.iloc[0:9, ]

,feature_1,feature_2,Correlation
125,ret_1d_shifted,sma_2d,0.999916
20154,sma_58d,sma_54d,0.998280
20009,sma_54d,sma_50d,0.998001
19864,sma_50d,sma_46d,0.997598
19719,sma_46d,sma_42d,0.997015
19574,sma_42d,sma_38d,0.996295
19429,sma_38d,sma_34d,0.995331
17692,sd_56,sd_59,0.994778
17402,sd_53,sd_56,0.994404


In [13]:
#| label: num-pca-components

#let's scale the data first 
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
print(x_train_scaled.shape)

pca = PCA()      # apply PCA with all the components
x_train_pca = pca.fit_transform(x_train_scaled)
x_train_pca.shape

cumul_var_ratio = np.cumsum(pca.explained_variance_ratio_)
print(cumul_var_ratio[1:30])

# to keep 95% of the variance 
num_comp = np.argmax(cumul_var_ratio > 0.95) + 1
print(f"{num_comp} components already explain 95% of the variance")

(1088, 144)
[0.37834386 0.42968913 0.45841187 0.48467234 0.50935192 0.5327829
 0.55369437 0.57437602 0.59469219 0.6148977  0.63469291 0.65415441
 0.6729319  0.69137148 0.70820194 0.72420138 0.73974372 0.75430529
 0.76717732 0.7799607  0.79140373 0.80250371 0.81319959 0.82292653
 0.83251898 0.84180221 0.85070096 0.8595149  0.86819361]
44 components already explain 95% of the variance


In [14]:
#| label: base-model-pca
#| warning: false

pca_levels = [0.8, 0.85, 0.9, 0.95, 0.99]
kernel_type = ['linear', 'rbf', 'poly', 'sigmoid']
df_xval_score = pd.DataFrame(columns = ['pca_levels','kernel_type','xval_score'])

tscv = TimeSeriesSplit(n_splits = 10, gap = 1)


for level in pca_levels: 
  for k_type in kernel_type: 
    model_svm_base = Pipeline([ 
      ('std_scaler', StandardScaler()),
      ('minmax_scaler', MinMaxScaler()), 
      ("pca", PCA(n_components = level, random_state = 42)), 
      ('classifier', SVC(kernel=k_type, probability=True, random_state=42))
      ]) 
    
    score_xval = cross_val_score(estimator = model_svm_base, 
                                 X = x_train, y = y_train, cv = tscv, 
                                 scoring = 'roc_auc', 
                                 n_jobs = 3).mean()
    new_row = pd.DataFrame({'pca_levels': [level], 
                            'kernel_type': [k_type], 
                            'xval_score': [score_xval]}) 
    df_xval_score = pd.concat([df_xval_score, new_row], ignore_index=True)
    
df_xval_score = df_xval_score.sort_values(by = 'xval_score', 
                                          ascending = False)

/var/folders/4k/sh_jysz50kq5c92jvljhs06c0000gp/T/ipykernel_96827/2973023535.py:24: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [15]:
#| label: gt_table-scores-pca
#| echo: false

df_xval_score.iloc[0:5, ]

,pca_levels,kernel_type,xval_score
18,0.99,poly,0.512301
2,0.80,poly,0.511581
6,0.85,poly,0.510526
14,0.95,poly,0.506918
10,0.90,poly,0.504158


In [16]:
num_comp = np.argmax(cumul_var_ratio > 0.8) + 1
print(f"{num_comp} components explain 80% of the variance")

23 components explain 80% of the variance


In [17]:
#| label: base-model-pca-fit
#| output: false

model_svm_base = Pipeline([
  ('std_scaler', StandardScaler()), 
  ('minmax_scaler', MinMaxScaler()), 
  ("pca", PCA(n_components=df_xval_score.iloc[0,0], random_state = 42)),
  ('classifier', SVC(kernel=df_xval_score.iloc[0,1], 
                     probability=True, random_state = 42))
])

model_svm_base.fit(x_train, y_train)

Pipeline(steps=[('std_scaler', StandardScaler()),
                ('minmax_scaler', MinMaxScaler()),
                ('pca', PCA(n_components=0.99, random_state=42)),
                ('classifier',
                 SVC(kernel='poly', probability=True, random_state=42))])

In [18]:
#| label: base-model-pca-metrics

acc_train = accuracy_score(y_train, model_svm_base.predict(x_train))
acc_test = accuracy_score(y_test, model_svm_base.predict(x_test))

print(np.round(acc_train, 4))
print(np.round(acc_test, 4))

0.7675
0.5055


In [19]:
#| label: confusion_matrix-basemodel-pca

conf_mat = ConfusionMatrixDisplay.from_estimator(model_svm_base, x_train, y_train, cmap = plt.cm.Blues)
plt.title('Confusion Matrix on Base Model')
plt.show()

<Figure size 640x480 with 2 Axes>

In [20]:
#| label: classification_report_pca

y_test_pred = model_svm_base.predict(x_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.52      0.90      0.66       143
           1       0.39      0.07      0.12       130

    accuracy                           0.51       273
   macro avg       0.45      0.49      0.39       273
weighted avg       0.46      0.51      0.40       273



In [21]:
#| label: roc_auc_pca

y_prob = model_svm_base.predict_proba(x_test)[:, 1]


fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print(np.round(roc_auc, 4))

plt.clf()
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

0.4798


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 1 Axes>

In [22]:
#| label: feat-sel-kbest

stat_test_type = [chi2, f_classif, mutual_info_classif]
kbest = [10, 15, 20, 25, 30, 35]
df_xval_score = pd.DataFrame(columns = ['stat_test', 'kbest','kernel_type','xval_score'])

feat_scaling = Pipeline([ 
  ('std_scaler', StandardScaler()), 
  ('minmax_scaler', MinMaxScaler())])
x_train_scaled = feat_scaling.fit_transform(x_train)
x_test_scaled = feat_scaling.fit(x_test)
  
for stt in stat_test_type: 
  for kb in kbest: 
    for k_type in kernel_type: 
      skbest = SelectKBest(score_func = stt, k = kb) 
      x_train_sel = skbest.fit_transform(x_train_scaled, y_train)
      #skbest.fit(x_train_scaled, y_train) 
      #x_skb = x_train_scaled.iloc[:, skbest.get_support()]
      mod_svc = SVC(kernel = k_type, probability = True) 
      score_skb = cross_val_score(mod_svc, x_train_sel, y_train, 
                                  cv=tscv, scoring='roc_auc', 
                                  n_jobs = 3).mean()
      new_row = pd.DataFrame({'stat_test': [stt.__name__], 
                              'kbest': [kb], 
                              'kernel_type': [k_type], 
                              'xval_score': [score_skb]})  
      df_xval_score=pd.concat([df_xval_score, new_row], ignore_index= True)
      
df_xval_score=df_xval_score.sort_values(by='xval_score', ascending=False)

/var/folders/4k/sh_jysz50kq5c92jvljhs06c0000gp/T/ipykernel_96827/3391194787.py:26: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [23]:
#| label: gt_table-scores-kbest
#| echo: false

df_xval_score.iloc[0:5]

,stat_test,kbest,kernel_type,xval_score
34,f_classif,20,poly,0.535852
0,chi2,10,linear,0.535781
28,f_classif,15,linear,0.533555
30,f_classif,15,poly,0.533029
32,f_classif,20,linear,0.526885


In [24]:
#| label: fit_kbest_train

skbest = SelectKBest(score_func = f_classif, k = df_xval_score.iloc[0,1]) 
skbest.fit(x_train_scaled, y_train)
x_train_skb = x_train.iloc[:, skbest.get_support()]
x_test_skb  = x_test.iloc[:, skbest.get_support()]

model_svm_reduc = Pipeline([
  ('scaler', MinMaxScaler()), 
  ('minmax_scaler', MinMaxScaler()), 
  ('svm',SVC(kernel = df_xval_score.iloc[0,2], probability = True))])

mod_svc_kbest = model_svm_reduc.fit(x_train_skb, y_train)

In [25]:
#| label: fit_kbest_test

y_prob = mod_svc_kbest.predict_proba(x_test_skb)[:,1]

fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

print(f" ROC AUC score for selected model is: {np.round(roc_auc, 4)}")

 ROC AUC score for selected model is: 0.4802


In [26]:
#| label: hierarchical-clust
#| fig-align: center

from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

cor = spearmanr(x_train).correlation
corr = (cor + cor.T)/2
np.fill_diagonal(corr, 1)
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))

plt.clf()
fig, ax = plt.subplots(figsize=(12, 8))
dendro = hierarchy.dendrogram(dist_linkage, 
                              labels=x_train.columns.to_list(), 
                              leaf_rotation=90, ax = ax)
plt.axhline(y=0.6, color='r', linestyle='--')
plt.show()

<Figure size 640x480 with 0 Axes>

<Figure size 1200x800 with 1 Axes>

In [27]:
#| label: hierarchical-clust-reg

from collections import defaultdict

cluster_ids = hierarchy.fcluster(dist_linkage, 0.5, criterion="distance")
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]
selected_features_names = df3.columns[selected_features]
print(selected_features_names.shape)
x_train_sel = x_train.iloc[:,selected_features]
x_test_sel = x_test.iloc[:, selected_features]

x_train_scaled = feat_scaling.fit_transform(x_train_sel)
x_test_scaled = feat_scaling.fit(x_test_sel)

mod_svm_l1 = LinearSVC(penalty="l1", dual=False, max_iter=10000)
mod_svm_l1.fit(x_train_scaled, y_train)
mod_svm_l2 = LinearSVC(penalty="l2")
mod_svm_l2.fit(x_train_scaled, y_train)

# Use SelectFromModel to select features based on the L1 regularization
model_svm_l1 = SelectFromModel(mod_svm_l1, prefit=True, max_features = 25)
x_train_l1 = x_train_sel.iloc[:,model_svm_l1.get_support()]

model_svm_l2 = SelectFromModel(mod_svm_l2, prefit=True,  max_features=25)
x_train_l2 = x_train_sel.iloc[:,model_svm_l2.get_support()]

(59,)


In [28]:
#| label: hierarchical-clust-fit

df_xval_score = pd.DataFrame(columns = ['reg_type','kernel_type','xval_score'])

for k_type in kernel_type: 
  pipe_emb = Pipeline([
    ('std_scaler', StandardScaler()), 
    ('scaler', MinMaxScaler()),  
    ('svm', SVC(kernel = k_type))
    ])
  score_emb_l1 = cross_val_score(pipe_emb, x_train_l1, y_train, 
                                 cv=tscv, scoring='roc_auc', 
                                 n_jobs = 3).mean() 
  score_emb_l2 = cross_val_score(pipe_emb, x_train_l2, y_train, 
                                 cv=tscv, scoring='roc_auc', 
                                 n_jobs = 3).mean()                                 
  new_row = pd.DataFrame({'reg_type': ['l1', 'l2'], 
                          'kernel_type': [k_type, k_type], 
                          'xval_score': [score_emb_l1, score_emb_l2]})  
  df_xval_score=pd.concat([df_xval_score, new_row], ignore_index= True)
  
df_xval_score = df_xval_score.sort_values(by='xval_score', ascending=False)

/var/folders/4k/sh_jysz50kq5c92jvljhs06c0000gp/T/ipykernel_96827/302261890.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [29]:
#| label: hierarchical-clust-auc

model_svm_emb = Pipeline([
  ('minmax_scaler', MinMaxScaler()), 
  ('std_scaler', StandardScaler()), 
  ('classifier', SVC(kernel = df_xval_score.iloc[0,1], 
                     probability = True, 
                     random_state = 42))
  ])
model_svm_emb.fit(x_train_l2, y_train)

x_test_l2 = x_test_sel.iloc[:,model_svm_l2.get_support()]
y_prob = model_svm_emb.predict_proba(x_test_l2)[:,1]

fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

print(f" ROC AUC score for selected model is: {np.round(roc_auc, 4)}")

 ROC AUC score for selected model is: 0.4514


In [30]:
#| label: hyper-param
#| eval: false

tscv = TimeSeriesSplit(n_splits = 10, gap = 1)

model_svm_tuned = Pipeline([
  #('Scaler', MinMaxScaler()), 
  ('std_scaler', StandardScaler()), 
  ("pca", PCA(n_components = 0.95)),
  ('clf', SVC(probability = True))
])

param_grid = {'clf__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20], 
              'clf__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 20], 
              'clf__kernel': ['rbf', 'poly', 'linear']
}

grid = GridSearchCV(model_svm_tuned, param_grid, verbose = 1, 
                    scoring = 'roc-auc', cv = tscv, n_jobs = 3)
grid.fit(x_train, y_train)

grid.best_params_
grid.best_score_
#grid.cv_results_

In [31]:
#| eval: false

svm_best_param = SVC(**grid.best_params_)
svm_best_param.fit(x_train, y_train, 
                   #eval_set = [(x_train, y_train), (x_test, y_test)], 
                   #verbose = True
                   )
#x_val_score = cross_val_score(svm_best_param, x_train, y_train, cv = tscv)

# what does this do?  
x_train_tuned = svm_best_param.decision_function(x_train)
x_val_score 

In [32]:
#| eval: false

y_pred = svm_best_param.predict(x_test)

acc_tuned_train = accuracy_score(y_train, svm_best_param.predict(x_train))
acc_tuned_test = accuracy_score(y_test, y_pred)

print(acc_train)
print(acc_test)